In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
23,model_9_8_9,0.905992,0.833521,0.886229,0.860085,0.017587,0.033595,0.011343,0.023124,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
24,model_9_8_8,0.905485,0.841307,0.892604,0.866927,0.017682,0.032024,0.010707,0.021993,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
27,model_9_8_7,0.904810,0.848797,0.898719,0.873505,0.017808,0.030513,0.010098,0.020906,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
29,model_9_8_6,0.903989,0.856079,0.904615,0.879886,0.017962,0.029043,0.009510,0.019851,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
31,model_9_8_5,0.902976,0.863044,0.910213,0.885979,0.018151,0.027638,0.008952,0.018844,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
34,model_9_8_4,0.901784,0.869732,0.915523,0.891808,0.018374,0.026288,0.008422,0.017881,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
35,model_9_8_3,0.900385,0.876062,0.920459,0.897302,0.018636,0.025011,0.007930,0.016973,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
38,model_9_8_2,0.898779,0.882058,0.924994,0.902465,0.018936,0.023801,0.007478,0.016120,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
40,model_9_8_1,0.896937,0.887639,0.929034,0.907220,0.019281,0.022674,0.007075,0.015334,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
43,model_9_8_0,0.894860,0.892828,0.932542,0.911570,0.019669,0.021627,0.006726,0.014615,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
